In [11]:
# libraries
import json
import time
from datetime import datetime
from Kafka import KafkaWriter, KafkaReader
from OpenWeatherMap import OpenWeatherMap

### Aufgabe 3 Buffern:
-> using topic "topic1" for buffering

In [12]:
# importing the classes
openWeatherMap = OpenWeatherMap()
# defining a topic specific writer and reader for kafka 
writer1 = KafkaWriter('topic1')
reader1 = KafkaReader('topic1')
# defining a counter
key = len(reader1.retrieve())

def format_date(dt: str) -> str:
    return datetime.fromtimestamp(float(dt)).strftime('%d.%m.%Y %H:%M')

# load 'locations.json' into a json-object and return it
def load_locations() -> json:
    with open('locations.json', mode='r') as file:
        return json.load(file)        

# for each location, query openWeatherMap for the 5-day forecast and 
# store the the returned values in Kafka
def collect_forecast_data(key) -> None:
    # getting the locations from the json through the load_locations() method
    locs = load_locations()
    # going over the locations
    for loc in locs:
        # reading the data from the api
        response = openWeatherMap.get_forecast(locs[loc])
        # iterating over the elements in the given dict
        for res in response['list']:
            # adding a city and timestamp attribute to the dictionary
            city = {'city' : response['city']['name'], 'coordinates':{'lat': locs[loc]['latitude'], 'lon': locs[loc]['longitude']}}
            res.update(city)
            # increasing the used counter
            key = key + 1
            # storing the data in kafka
            writer1.store(str(key), res)        

### Aufgabe 4 Dubletten:
-> using topic "weather.forecast" as "clean" topic with the eliminated duplicates

In [13]:
# defining kafka producer for the "clean" topic
writer2 = KafkaWriter('weather.forecast')

# method for eliminating the duplicates from the test-topic and storing the clean data in a new topic
def cleaning() -> None:  
    # getting all 'raw' data from the topic 'topic1'
    all_data = reader1.retrieve()
    
    # getting the already exisiting 'filtered' data in 'weather.forecast'
    existing_data = KafkaReader('weather.forecast').retrieve()
    # defining a counter for the keys of the data in kafka
    counter = len(existing_data)
    
    # looping through the 'raw' data, that means over every dictionary-element of it
    for a in all_data:
        # checking if existing_data is empty
        if not existing_data:
            # increase counter
            counter = counter + 1
            # if existing_data is empty store the first element of a in existing_data and append it to it for further checking
            writer2.store(str(counter), a)
            existing_data.append(a)
        else:
            # loop through the existing_data in the clean topic
            for e in existing_data:
                # check if a is already part of the clean topic
                if a not in existing_data:
                    # if it isn't already in the clean topic, increase counter, store it to kafka and append it to the list
                    counter = counter + 1
                    writer2.store(str(counter), a)
                    existing_data.append(a)                    

In [14]:
# first reading in the data from the api with collect_forecast_data every 15 min
# then reading the buffered data from 'topic1' and writing the unique values to the clean topic 'weather.forecast'
while True:
    collect_forecast_data(key)
    cleaning()
    time.sleep(900) 

Error sending OffsetCommitRequest_v2 to node coordinator-1 [Cancelled: <BrokerConnection node_id=coordinator-1 host=kafka-1:19092 <connected> [IPv4 ('172.18.0.5', 19092)]>]
Marking the coordinator dead (node coordinator-1) for group 4d8a9a5c9086466fa5c4a64fc6e7ef97: Cancelled: <BrokerConnection node_id=coordinator-1 host=kafka-1:19092 <connected> [IPv4 ('172.18.0.5', 19092)]>.


KeyboardInterrupt: 

<b>Idee des Ansatzes:</b> \
Da nicht ganz klar ist, wie oft die Wetterdaten von OpenWeatherMap aktualisiert werden, werden die Dictionary Elemente in beiden Topics miteinander verglichen. Dadurch ist gewährleistet, dass es keine Dubletten gibt. Jedoch wenn der Forecast in OpenWeatherMap aktualisiert wird und sich bspw. die Temperatur für eine Vorhersage ändert bzw. genauer wird, gilt dies nicht mehr als Dublette, sondern als aktualisierter Forecast und wird in Folge dessen auch in das cleane Topic geschrieben. Die doppelten Vorhersagen werden jedoch im Rahmen der get_dataframe() Methode vor der weiteren Verarbeitung herausgefiltert.